Tạo một datafram mới để chứa thông tin làm bảng RFM


In [1]:
import pandas as pd

In [2]:
from datetime import datetime

In [3]:
# Tạo dataframe mới để lưu bảng RFM
RFM_table = pd.DataFrame()

In [4]:
# Nạp file 'Ontop_General_Records_cleaned.csv' đã được làm sạch trước đó
GRC = pd.read_csv("Ontop_General_Records_cleaned.csv")

In [5]:
# Chuyển cột Date thành định dạng datetime
GRC['DATE'] = pd.to_datetime(GRC['DATE'])

In [6]:
# Chuyển cột Amount thành định dạng float
GRC['Amount'] = GRC['Amount'].str.replace(",", "")
GRC['Amount'] = GRC['Amount'].astype(float)

In [7]:
GRC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     6012 non-null   int64         
 1   DATE           6012 non-null   datetime64[ns]
 2   Record_Type    6012 non-null   object        
 3   PAYEE          6012 non-null   object        
 4   Payee_ID       6012 non-null   object        
 5   Internal Code  5195 non-null   object        
 6   Description    6012 non-null   object        
 7   Amount         6012 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 375.9+ KB


In [8]:
# Tạo cột date_diff để tính chênh lệch ngày với ngày 30-11-2022 (ngày cuối cùng của tháng có dữ liệu)
GRC['Date_diff'] = datetime(2022,11,30) - GRC['DATE']

In [9]:
# Tính các cột để làm cơ sở tính RFM
RFM_table = GRC[GRC['Record_Type'] == "REV"].groupby('Payee_ID').agg({'Date_diff':"min" , 'Internal Code':'nunique' , 'Amount':'sum'}).reset_index()

In [ ]:
# Ra được bảng như bên dưới, nhưng sẽ đổi tên lại cho dễ hiểu
RFM_table

In [10]:
# Đổi lại tên cột
RFM_table = RFM_table.rename(columns={'Internal Code':'Customer_Order', 'Amount':'Customer_Revenue', 'Date_diff':'Customer_Last_Order' })

In [11]:
# Customer_Order == Frequency
# Customer_Revenue == Monetary
# Customer_Last_Order == Recency
RFM_table

,Payee_ID,Customer_Last_Order,Customer_Order,Customer_Revenue
0,ABCO,946 days,1,5091.60
1,ADCA,745 days,1,6215.00
2,ANVA,54 days,42,269871.53
3,ARCU,250 days,3,42514.23
4,ATOS,183 days,3,16295.75
...,...,...,...,...
60,XEMU,1021 days,2,9333.51
61,XOTU,55 days,14,10166.95
62,YULH,30 days,81,1662235.04
63,ZAYN,33 days,18,193473.58


In [12]:
# tạo các cột R, F, M tương ứng
# F_score phải chia 8 rồi drop dup;icate 3 vì ở 3 Quartile đầu tiên, giá trị bị trùng, làm vậy mới ra được 5 phần
RFM_table['R_score'] = pd.qcut(RFM_table['Customer_Last_Order'], q=5, labels=False)
RFM_table['F_score'] = pd.qcut(RFM_table['Customer_Order'], q=8, labels=False, duplicates="drop")
RFM_table['M_score'] = pd.qcut(RFM_table['Customer_Revenue'], q=5, labels=False, precision = 2)

In [13]:

# Thay đổi cột R_score, vì ngày càng lớn thì hạng phải càng thấp
# Cũng như chuyển từ dãy 0-4 sang dãy 1-5 cho các Score
RFM_table['R_score'] = RFM_table['R_score'].replace({0:5 , 1:4 , 2:3 , 3:2 , 4:1})
RFM_table['M_score'] = RFM_table['M_score'].replace({0:1 , 1:2 , 2:3 , 3:4 , 4:5})
RFM_table['F_score'] = RFM_table['F_score'].replace({0:1 , 1:2 , 2:3 , 3:4 , 4:5})

In [14]:
# Nối các điểm R, F, M lại
RFM_table["RFM_score"] = RFM_table['R_score'].astype(str) + RFM_table['F_score'].astype(str) + RFM_table['M_score'].astype(str)

In [15]:
# nhập bảng Rank_RFM
Rank_RFM = pd.read_csv("Rank_RFM.csv")

In [16]:
# chuyển định dạng cột Score để có thể nối hai file với nhau
Rank_RFM['Scores'] = Rank_RFM['Scores'].astype(str)

In [17]:
RFM_final = pd.merge(RFM_table , Rank_RFM , how="left" , left_on='RFM_score' , right_on="Scores")

In [18]:
RFM_final

,Payee_ID,Customer_Last_Order,Customer_Order,Customer_Revenue,R_score,F_score,M_score,RFM_score,Segment,Scores
0,ABCO,946 days,1,5091.60,1,1,2,112,Lost customers,112
1,ADCA,745 days,1,6215.00,2,1,3,213,About To Sleep,213
2,ANVA,54 days,42,269871.53,5,5,5,555,Champions,555
3,ARCU,250 days,3,42514.23,3,2,4,324,Need Attention,324
4,ATOS,183 days,3,16295.75,3,2,4,324,Need Attention,324
...,...,...,...,...,...,...,...,...,...,...
60,XEMU,1021 days,2,9333.51,1,1,3,113,Cannot Lose Them,113
61,XOTU,55 days,14,10166.95,5,4,3,543,Loyal,543
62,YULH,30 days,81,1662235.04,5,5,5,555,Champions,555
63,ZAYN,33 days,18,193473.58,5,4,5,545,Champions,545


In [19]:
RFM_final.to_csv('RFM_final.csv')